In [1]:
import torch
import torch.nn as nn

## Пример прямого и обратного распространения

In [3]:
class TransparentNet(nn.Module):

    def __init__(self, w: torch.Tensor, b: torch.Tensor):

        super().__init__()
        self.w = nn.Parameter(w)
        self.b = nn.Parameter(b)
        self.softmax = nn.Softmax(dim=-1)


    def forward(self, x):

        self.z1 = self.w @ x + self.b
        self.z1.retain_grad()

        self.z2 = self.softmax(self.z1)
        self.z2.retain_grad()

        return self.z2

w = torch.tensor([[0.2, 0.4, 0.9],\
                  [0.1,-0.2,-0.5]])

b = torch.tensor([0.0, 0.2]).T

x = torch.tensor([0.3, 0.3, 0.9]).T

y = torch.tensor([0.0, 1.0])

net = TransparentNet(w=w, b=b)

print('Forward')
y_p = net(x)
dy = y - y_p
y_p.retain_grad()

loss = torch.pow(dy, 2).sum()
loss.retain_grad()

print('Backward')
loss.backward()

print(f'Loss: {loss}')
print(f'loss_grad: {loss.grad}')
print(f'y_p_grad: {y_p.grad}')
print(f'z2_grad: {net.z2.grad}')
print(f'z1_grad: {net.z1.grad}')
print(f'w_grad: {net.w.grad}')
print(f'b_grad: {net.b.grad}')

Forward
Backward
Loss: 1.2191184759140015
loss_grad: 1.0
y_p_grad: tensor([ 1.5615, -1.5615])
z2_grad: tensor([ 1.5615, -1.5615])
z1_grad: tensor([ 0.5346, -0.5346])
w_grad: tensor([[ 0.1604,  0.1604,  0.4811],
        [-0.1604, -0.1604, -0.4811]])
b_grad: tensor([ 0.5346, -0.5346])


### Автоматическое дифференцирование в PyTorch

Автоматическое дифференцирование в основе своей просто:
1. Правило композиции определяет, как перейти от одного слоя (математической операции) к другому;
2. Строится динамический граф вычислений;
3. Слои (математические операции) дифференцируются благодаря тому, что для всех элементарных операций определены методы вычисления производных;

Реализованный выше и разобранный на занятии пример вполне иллюстрирует все эти этапы.

*НЛО*:
В тензорном исчислении, тензор можно представить следующей формулой:

$$ \texttt{тензор} = \text{линейный оператор} + \text{система координат} + \text{закон его преобразования при замене координат} $$

Тензоры в `pytorch` не являются тензорами в этом смысле слова. Но "формула" `torch.tensor` очень похожа, потому заимствование этого названия вполне уместно:

$$ \texttt{torch.tensor} = \text{линейный оператор} + \text{граф вычислений} + \text{метод дифференцирования} $$